# Synthetic Dataset Generation using RAG

- Author: [Jaehun Choi](https://github.com/ash-hun)
- Design: 
- Peer Review: [syshin0116](https://github.com/syshin0116), [Kane](https://github.com/HarryKane11)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

This tutorial covers an example of generating a synthetic dataset using RAG. Typically, it is used to create evaluation datasets for Domain Specific RAG pipelines or to generate synthetic data for model training. This tutorial will focus on the following two features. While the structure is the same, their intended use and purpose differ.

**Features**

- Domain Specific RAG Evaluation Dataset : Generates a domain specific synthetic dataset (Context, Question, Answer) for evaluating the RAG pipeline.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Domain Specific RAG Evaluation Dataset](#domain-specific-rag-evaluation-dataset)


### References

- [autoRAG github](https://github.com/Marker-Inc-Korea/AutoRAG?tab=readme-ov-file#3-qa-creation)
- [ragas github : singlehop question](https://github.com/explodinggradients/ragas/blob/main/src/ragas/testset/synthesizers/single_hop/prompts.py)
- [huggingface : RAG Evaluation Dataset Prompt](https://huggingface.co/datasets/Ash-Hun/Create_RAG_Evalauation_Data)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Synthetic Dataset Generation using RAG",
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [5]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Domain Specific RAG Evaluation Dataset

Generates a synthetic dataset (Context, Question, Answer) for evaluating the Domain Specific RAG pipeline.

- Context: A context randomly selected from documents in a specific domain is used as the ground truth.
- Question: A question that can be answered using the context.
- Answer: An answer generated based on the context and the question.

In [6]:
# Import Library
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

### Question Generating Prompt

A prompt for generating questions from a given context using the RAG (Retriever Augmented Generation) technique is structured as follows.  
It consists of four main sections—Instruction, Requirements, Style, and Example—along with an Indicator section where actual variable values are mapped. Each section is explained below: 
- Instruction: Provides overall guidance for the prompt, including the purpose of the task and an explanation of the structured prompt sections.
- Requirements: Lists essential conditions that must be met when performing the task.
- Style: Specifies the stylistic guidelines for the generated output.
- Example: Includes actual execution examples.

In [7]:
Q_GEN = """
[ Instruction ] : 
- Your mission is to generate detailed ONE QUESTION that can yield correct answers from the GIVEN CONTEXT.
- When creating a QUESTION, you should carefully consider the following items:
  - Requirements : Essential requirements that must be included
  - Style : The form and style of the generated question
  - Think : Elements and procedures you need to self-examine for the created question

<Requirements>
- The questions you generate must always maintain high quality.
- Please do not print and generate any other unnecessary words.
- The Questions are created from the given context, but it must be created with an appropriate balance between general content and domain-specific content.
- If the given context related figure, you must generate the question related figure data.
- Finally, verify that the generated question contains only ONE QUESTION itself without any unnecessary description or content.
</Requirements> 

<Style>
- The expressions you use should either be inferred from the given context or be directly used expressions.
- Any expressions involving formulas must always be enclosed within $ symbols.
- Text that is conceptually represented with subscripts or superscripts should be expressed as mathematical formulas.
- You should compose questions that are as natural as possible within the context.
</Style>


Now, It's your turn. You must generate long and detailed high-quality questions from the given context while following the mentioned <Requirements> and <Style>.
The examples below consist of positive samples and negative samples. Please refer to the given examples to generate your answer.
When you generated QUESTION, you should take a deep breath and think step-by-step and generate the most natural Korean question.

<Example>
 - chunk : 
1) 공정(설비) 정의 및 특징

● 사출성형 정의
- 사출성형이란 플라스틱 성형법 중의 한 방법으로서, 열가소성 수지를 가열해서 유동 상태로 되었을 때 금형의 공동부(Cavity, 이하 ‘캐비티‘)에 가압 주입하여 금형 내에서 냉각시킴으로써, 금형의 공동부에 상당하는 성형품을 만드는 방법이다.

● 사출성형 특징
① 성형 사이클이 짧고, 성형 능률이 좋다. (대량 생산 가능) 
② 광범위한 수지의 성형 (열가소성 수지, 열경화성 수지) 
③ 자동제어화가 가능하다. 
④ 현재 대형 성형품의 성형 가능하다. 
⑤ 정밀도가 높아 복잡한 모양 가능하다.

- positive sample : 
    - 사출성형의 과정은 자동제어화가 가능한가?
    - 사출성형의 정의란 무엇인가?
    - 사출성형의 특징이 무엇인가?
    - 사출성형 과정에서 대량생산이 가능한 경우엔 어떠한 특징을 지니는가?

- negative sample :
    - 사출성형의 정의와 특징을 고려할 때, 사출성형의 성형 사이클과 성형 능률의 특징은 무엇이며, 이 특징으로 인해 어떤 생산이 가능해지는가?
    - 사출성형 정의와 특징을 설명하고, 사출성형의 성형 사이클이 짧고 성형 능률이 좋은 이유를 설명해 주세요. 
    - 사출성형의 자동제어화 가능성과 정밀도에 대해 설명해 주세요.
    - 사출성형의 정의와 특징을 바탕으로, 사출성형의 공정(설비)에서 자동제어화와 정밀도가 중요한 이유를 설명해 주세요.
</Example>

- Given context : {context}
"""

In [8]:
# Step 01. Generation Question from Domain Specific Context

def question_generate(context:list, q_gen_prompt:str=Q_GEN) -> str:
    # Create an OpenAI object
    model = ChatOpenAI(temperature=0, model_name="gpt-4o")
    
    # Set up the prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI assistant"),
            ("user", q_gen_prompt),
        ]
    )
    
    # Combine the prompt, model, and JsonOutputParser into a chain
    chain = prompt | model
    
    # Prepare inputs for batch processing
    inputs = [{"context": ctx} for ctx in context]

    results = chain.batch(inputs)
    return [result.content for result in results]

In [9]:
sample_context = [
    "강관을 제조방법별로 분류하면 접합부의 유무에 따라 무계목강관과 접합강관으로 크게 나누어집니다. 무계목 강관 빌릿 등의 소재를 가열해 천공기로 중심에 구멍을 뚫고 중공소재로 만든 후 압연하거나 인발기 등로 인발해 소정이 외경과 두께로 만든 강관을 말하며소재에 구멍을 뚫은 다음 연신하는 경사압연방식과 이미 천공된 소재를 프레스로 압출하는 프레스방식등로 만듭니다. 예 심레스강관 접합 강관 - 단접강관용접강관용접법과 제조공정에 따라 세분화한다. 단접강관 소정의 폭으로 절단한 스켈프강관의 재로가 되는 반제품를 약 1400 ºC 정도로 가열한 후 연속식 단접기에서 관형태로 성형 동시에 양단을 롤로 압착시켜 제조합니다. 대부분이 소경관이며 대량 생산방식이므로 다품종 소량생산에는 어려움이 있습니다. 용접강관 대강 또는 스켈프를 상온에서 연속 롤 성형 통해 관형태로 성형 후 접합부를 용접해 제조하는 방법으로서 용접법에 따라 전기저항용접강관ERW강관가스용접관아크용접강관SAW강관으로 분류하며 SAW강관은 성형 따라 UOE 프레스강관스파이럴 강관롤벤딩 강관연속성형 구분합니다. UOE강관은 강력한 U형 프레스로 상온에서 전체길이를 U자형으로 구부리고O형 프레스로 성형 확관합니다. 장척 대경의 용접관 제조에 적합하고 성형 단순하며 용접길이가 짧아 능률이 높다는 특징이 있습니다. 스파이럴 조관법은 광폭 대강을 나선형으로 감아 원통형으로 성형 뒤 맞닿은 부분을 용접해 제조합니다. 동일폭의 강대를 사용해 나선 각도성형 각도를 변경함으로써 구경이 다른 강관을 제조할 수 있고 용접선이 나선형이기 때문에 응력이 비교적 낮습니다. 롤벤딩에 의한 조관법은 옛날부터 널리 사용된 방법이며 강관을 소경의 길이로 절단해 3~5개의 롤로 곡면 성형 원통형으로 만든뒤 맞닿은 양 edge를 용접하는 방법입니다.", 
    "Joining은 두 개 이상의 재료를 하나의 단위로 연결하는 과정입니다. 이 과정은 용접, 리벳, 브레이징, 접착 결합 등 다양한 기법을 통해 이루어지며, 사용되는 재료와 결합 부위의 강도에 따라 적합한 방법이 선택됩니다. 제조업과 건설업에서 Joining은 최종 제품의 구조적 안정성, 내구성 및 기능성을 보장하는 중요한 과정입니다. 각 Joining 방법은 장단점이 있으며, 온도, 압력, 재료의 호환성 등 여러 요소가 적합한 기법을 결정하는 데 중요한 역할을 합니다. 자동차, 항공 우주, 건설 산업 등에서 적용되며 오랜 시간 사용할 수 있는 제품을 만드는 데 필수적인 작업입니다."
]
sample_question = question_generate(context=sample_context)
sample_question

['무계목 강관과 접합 강관의 제조 방법을 비교할 때, 무계목 강관의 제조 과정에서 사용되는 경사압연방식과 프레스방식의 차이점은 무엇인가?',
 'Joining 과정에서 사용되는 다양한 기법 중, 용접과 브레이징의 차이점은 무엇이며, 이 두 기법이 각각 어떤 상황에서 적합한지 설명할 수 있는가?']

### Question Evolving Prompt


A prompt that utilizes the RAG (Retriever Augmented Generation) technique to correct inaccurate information or generate more evolved questions based on a given context and a draft question is structured as follows.  
It consists of four main sections—Instruction, Evolving, and Example along with an Indicator section where actual variable values are mapped. Each section is explained below:  

- Instruction: Provides the overall guidance for the prompt.
- Evolving: Contains step-by-step instructions to achieve the purpose of the prompt.
- Example: Includes actual execution examples.

In [10]:
Q_EVOLVE="""
[ Instruction ] : 
- Your mission is to review & check the context and the question to determine whether the answer to the question can be obtained from the given context, and then evolve the following question.
- If the question can be fully answered using information from the given context, please return the same question.
- If the question cannot be answered using information from the given context, please modify the question.
- This process is called <Evolving> and should be carried out according to the procedure below, ultimately returning either the original or the EVOLVING QUESTION.

<Evolving>
1. Understand the given context and question.
2. Determine whether all the information required to answer the question is present in the context.
  2-1. If you conclude that all necessary information is available in the context:
    2-1-1. Treat the given question as the EVOLVING QUESTION.
  2-2. If you conclude that not all necessary information is present in the context:
    2-2-1. Modify the form or content of the question so that it can be answered using only the information provided in the context.
    2-2-2. Treat the modified question as the EVOLVING QUESTION.
3. Return the evolving question.
</Evolving> 

You can refer to the following examples when performing the <Evolving> process, however you must never explain the <Evolving> process. Only provide the EVOLVING QUESTION.
Take sufficient time to think logically through each step as you proceed. 
If you create a high-quality EVOLVING QUESTION, you may receive a small tip. 
Now it's your turn. Take a deep breath and start!

<Example>
 - chunk : 
1) 공정(설비) 정의 및 특징

● 사출성형 정의
- 사출성형이란 플라스틱 성형법 중의 한 방법으로서, 열가소성 수지를 가열해서 유동 상태로 되었을 때 금형의 공동부(Cavity, 이하 ‘캐비티‘)에 가압 주입하여 금형 내에서 냉각시킴으로써, 금형의 공동부에 상당하는 성형품을 만드는 방법이다.

● 사출성형 특징
① 성형 사이클이 짧고, 성형 능률이 좋다. (대량 생산 가능) 
② 광범위한 수지의 성형 (열가소성 수지, 열경화성 수지) 
③ 자동제어화가 가능하다. 
④ 현재 대형 성형품의 성형 가능하다. 
⑤ 정밀도가 높아 복잡한 모양 가능하다.

- question : 사출성형 정의와 특징을 설명하고, 사출성형의 성형 사이클이 짧고 성형 능률이 좋은 이유를 설명해 주세요.

- EVOLVING QUESTION : 사출성형 정의와 특징을 설명하고, 그중 대량 생산을 가능하게 하는 이유가 어떤 것인지 알려주세요.
</Example>

- Given context : {context}
- Question :{question}
- EVOLVING QUESTION :
"""

In [11]:
# Step 02. Evolving Question from Domain Specific Context & Question
def question_evolving(context:str, question:str, evolving_prompt:str=Q_EVOLVE) -> str:
    # Create an OpenAI object
    model = ChatOpenAI(temperature=0, model_name="gpt-4o")
    
    # Set up the prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI assistant"),
            ("user", evolving_prompt),
        ]
    )
    
    # Combine the prompt, model, and JsonOutputParser into a chain
    chain = prompt | model

    # Prepare inputs for batch processing
    inputs = [{"context": ctx, "question": q} for ctx, q in zip(context, question)]

    results = chain.batch(inputs)
    return [result.content for result in results]

In [12]:
evolved_question = question_evolving(context=sample_context, question=sample_question)
evolved_question

['무계목 강관의 제조 과정에서 사용되는 경사압연방식과 프레스방식의 차이점은 무엇인가?',
 'Joining 과정에서 사용되는 다양한 기법 중, 용접과 브레이징의 차이점에 대한 정보는 주어지지 않았습니다. 따라서, 이 두 기법이 각각 어떤 상황에서 적합한지에 대한 질문으로 수정하겠습니다. \n\nJoining 과정에서 사용되는 다양한 기법 중, 어떤 요소들이 적합한 기법을 결정하는 데 중요한 역할을 하는지 설명할 수 있는가?']

### Answer Generating Prompt

A prompt that uses the RAG (Retriever Augmented Generation) technique to generate the final answer based on a given context and question is structured as follows. It consists of two main sections—Instruction and Example—along with an Indicator section where actual variable values are mapped. Each section is explained below:  

- Instruction: Provides the overall guidance for the prompt.
- Example: Includes actual execution examples.

In [13]:
A_GEN = """
[ Instruction ] : 
- Your mission is to produce an accurate answer by reviewing the provided CONTEXT and QUESTION.
- When creating your ANSWER, you must use the information from the CONTEXT and strive to make your response as detailed and high-quality as possible.
- Please refer to the <Example> below to create your ANSWER.

<Example>
 - CONTEXT : 
1) 공정(설비) 정의 및 특징

● 사출성형 정의
- 사출성형이란 플라스틱 성형법 중의 한 방법으로서, 열가소성 수지를 가열해서 유동 상태로 되었을 때 금형의 공동부(Cavity, 이하 ‘캐비티‘)에 가압 주입하여 금형 내에서 냉각시킴으로써, 금형의 공동부에 상당하는 성형품을 만드는 방법이다.

● 사출성형 특징
① 성형 사이클이 짧고, 성형 능률이 좋다. (대량 생산 가능) 
② 광범위한 수지의 성형 (열가소성 수지, 열경화성 수지) 
③ 자동제어화가 가능하다. 
④ 현재 대형 성형품의 성형 가능하다. 
⑤ 정밀도가 높아 복잡한 모양 가능하다.

- QUESTION : 사출성형 정의와 특징을 설명하고, 그중 대량 생산을 가능하게 하는 이유가 어떤 것인지 알려주세요.

- ANSWER : 사출성형은 플라스틱 성형법의 한 종류로 열가소성 수지를 가열해서 유동 상태로 되었을 때 금형의 공동부(Cavity, 이하 '캐비티')에 가압 주입하여 금형 내에서 냉각시킴으로써, 금형의 공동부에 상당하는 성형품을 만드는 방법을 의미합니다. 이러한 사출성형은 대표적으로 자동제어화가 가능하다는 점, 대형 성형품의 성형이 가능하다는 점, 높은 정밀도를 이용한 복잡한 모양의 성형도 가능하다는 점 등 여러 특징을 가지고 있습니다. 특히 성형 사이클이 짧고, 성형 능률이 좋다는 특징은 대량 생산을 가능하게 합니다.
</Example>

Take sufficient time to think logically through each step as you proceed. 
If you create a detail high-quality ANSWER, you may receive a small tip. 
Now it's your turn. Take a deep breath and start!

- CONTEXT : {context}

- QUESTION :{question}

- ANSWER :

"""

In [14]:
# Step 03. Generate Answer from Domain Specific Context & Evolved Question
def answer_generate(context:str, evolved_question:str, a_gen_prompt:str=A_GEN) -> str:
    # Create an OpenAI object
    model = ChatOpenAI(temperature=0, model_name="gpt-4o")
    
    # Set up the prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI assistant"),
            ("user", a_gen_prompt),
        ]
    )
    # Combine the prompt, model, and JsonOutputParser into a chain
    chain = prompt | model

    # Prepare inputs for batch processing
    inputs = [{"context": ctx, "question": q} for ctx, q in zip(context, evolved_question)]

    results = chain.batch(inputs)
    return [result.content for result in results]

In [15]:
answer = answer_generate(context=sample_context, evolved_question=evolved_question)
answer

['무계목 강관의 제조 과정에서 사용되는 경사압연방식과 프레스방식의 차이점은 다음과 같습니다:\n\n1. **경사압연방식**:\n   - **과정**: 이 방식은 소재에 구멍을 뚫은 다음, 연신하는 과정을 포함합니다. 즉, 빌릿 등의 소재를 가열하여 천공기로 중심에 구멍을 뚫고, 중공소재로 만든 후 압연하거나 인발기 등을 사용하여 소정의 외경과 두께로 강관을 만듭니다.\n   - **특징**: 경사압연방식은 소재를 천공한 후 연신하여 원하는 형태로 만드는 방식으로, 주로 압연과 인발을 통해 강관의 외경과 두께를 조절합니다.\n\n2. **프레스방식**:\n   - **과정**: 이 방식은 이미 천공된 소재를 프레스를 사용하여 압출하는 방식입니다. 즉, 중공소재를 프레스로 압출하여 강관을 제조합니다.\n   - **특징**: 프레스방식은 천공된 소재를 프레스를 통해 직접 압출하여 강관을 만드는 방식으로, 주로 프레스의 힘을 이용하여 소재를 성형합니다.\n\n이 두 방식의 주요 차이점은 경사압연방식이 천공 후 연신을 통해 강관을 성형하는 반면, 프레스방식은 천공된 소재를 프레스로 직접 압출하여 성형한다는 점입니다.',
 'In the Joining process, several factors play a crucial role in determining the most suitable technique for connecting materials. These factors include:\n\n1. **Material Compatibility**: The type of materials being joined is a primary consideration. Different materials may require specific joining methods due to their physical and chemical properties. For instance, metals might be more suitable for welding, while cerami

### Overall Code

In [34]:
class SyntheticGenerator():
    def __init__(self, data:list|None=None, Gen_prompt_question:str=Q_GEN, Gen_prompt_evolving:str=Q_EVOLVE, Gen_prompt_answer:str=A_GEN)->None:
        self.data = data
        self.q_prompt = Gen_prompt_question
        self.e_prompt = Gen_prompt_evolving
        self.a_prompt = Gen_prompt_answer

    def _generate_question(self, data:list[str]) -> list:
        # Create an OpenAI object
        model = ChatOpenAI(temperature=0, model_name="gpt-4o")
        
        # Set up the prompt template
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You are a helpful AI assistant"),
                ("user", self.q_prompt),
            ]
        )
        
        # Combine the prompt, model, and JsonOutputParser into a chain
        chain = prompt | model
        
        # Prepare inputs for batch processing
        inputs = [{"context": ctx} for ctx in data]

        results = chain.batch(inputs)
        return [[ctx, result.content] for ctx, result in zip(data, results)]

    def _evolving_question(self, data:list[list[str]]) -> list[list[str]]:
        # Create an OpenAI object
        model = ChatOpenAI(temperature=0, model_name="gpt-4o")
        
        # Set up the prompt template
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You are a helpful AI assistant"),
                ("user", self.e_prompt),
            ]
        )
        
        # Combine the prompt, model, and JsonOutputParser into a chain
        chain = prompt | model

        # Prepare inputs for batch processing
        inputs = [{"context": each[0], "question": each[1]} for each in data]

        results = chain.batch(inputs)
        return [[each[0], result.content] for each, result in zip(data, results)]

    def _generate_answer(self, data:list[list[str]]) -> list[list[str]]:
        # Create an OpenAI object
        model = ChatOpenAI(temperature=0, model_name="gpt-4o")
        
        # Set up the prompt template
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You are a helpful AI assistant"),
                ("user", self.a_prompt),
            ]
        )
        # Combine the prompt, model, and JsonOutputParser into a chain
        chain = prompt | model

        # Prepare inputs for batch processing
        inputs = [{"context": ctx, "question": q} for ctx, q in data]

        results = chain.batch(inputs)
        return [[each[0], each[1], result.content] for each, result in zip(data, results)]

    def run(self, data:list|None=None, save_path:str|None=None) -> list:
        raw_data = data if self.data is None else self.data
        print(raw_data)
        if raw_data is None:
            raise ValueError("Empty Data")
        else:
            ctx_data_q = self._generate_question(data=raw_data)
            ctx_data_eq = self._evolving_question(data=ctx_data_q)
            ctx_data_eq_a = self._generate_answer(data=ctx_data_eq)
        
        if save_path is None:
            return ctx_data_eq_a
        else:
            import pandas as pd
            pd.DataFrame(ctx_data_eq_a, columns=['context', 'question', 'answer']).to_csv(save_path, index=False)
            return ctx_data_eq_a


In [35]:
sample_context = [
    "강관을 제조방법별로 분류하면 접합부의 유무에 따라 무계목강관과 접합강관으로 크게 나누어집니다. 무계목 강관 빌릿 등의 소재를 가열해 천공기로 중심에 구멍을 뚫고 중공소재로 만든 후 압연하거나 인발기 등로 인발해 소정이 외경과 두께로 만든 강관을 말하며소재에 구멍을 뚫은 다음 연신하는 경사압연방식과 이미 천공된 소재를 프레스로 압출하는 프레스방식등로 만듭니다. 예 심레스강관 접합 강관 - 단접강관용접강관용접법과 제조공정에 따라 세분화한다. 단접강관 소정의 폭으로 절단한 스켈프강관의 재로가 되는 반제품를 약 1400 ºC 정도로 가열한 후 연속식 단접기에서 관형태로 성형 동시에 양단을 롤로 압착시켜 제조합니다. 대부분이 소경관이며 대량 생산방식이므로 다품종 소량생산에는 어려움이 있습니다. 용접강관 대강 또는 스켈프를 상온에서 연속 롤 성형 통해 관형태로 성형 후 접합부를 용접해 제조하는 방법으로서 용접법에 따라 전기저항용접강관ERW강관가스용접관아크용접강관SAW강관으로 분류하며 SAW강관은 성형 따라 UOE 프레스강관스파이럴 강관롤벤딩 강관연속성형 구분합니다. UOE강관은 강력한 U형 프레스로 상온에서 전체길이를 U자형으로 구부리고O형 프레스로 성형 확관합니다. 장척 대경의 용접관 제조에 적합하고 성형 단순하며 용접길이가 짧아 능률이 높다는 특징이 있습니다. 스파이럴 조관법은 광폭 대강을 나선형으로 감아 원통형으로 성형 뒤 맞닿은 부분을 용접해 제조합니다. 동일폭의 강대를 사용해 나선 각도성형 각도를 변경함으로써 구경이 다른 강관을 제조할 수 있고 용접선이 나선형이기 때문에 응력이 비교적 낮습니다. 롤벤딩에 의한 조관법은 옛날부터 널리 사용된 방법이며 강관을 소경의 길이로 절단해 3~5개의 롤로 곡면 성형 원통형으로 만든뒤 맞닿은 양 edge를 용접하는 방법입니다.", 
    "Joining은 두 개 이상의 재료를 하나의 단위로 연결하는 과정입니다. 이 과정은 용접, 리벳, 브레이징, 접착 결합 등 다양한 기법을 통해 이루어지며, 사용되는 재료와 결합 부위의 강도에 따라 적합한 방법이 선택됩니다. 제조업과 건설업에서 Joining은 최종 제품의 구조적 안정성, 내구성 및 기능성을 보장하는 중요한 과정입니다. 각 Joining 방법은 장단점이 있으며, 온도, 압력, 재료의 호환성 등 여러 요소가 적합한 기법을 결정하는 데 중요한 역할을 합니다. 자동차, 항공 우주, 건설 산업 등에서 적용되며 오랜 시간 사용할 수 있는 제품을 만드는 데 필수적인 작업입니다."
]

generator = SyntheticGenerator(data=sample_context)
synthetic_data = generator.run(save_path='./sample_data.csv')
synthetic_data

['강관을 제조방법별로 분류하면 접합부의 유무에 따라 무계목강관과 접합강관으로 크게 나누어집니다. 무계목 강관 빌릿 등의 소재를 가열해 천공기로 중심에 구멍을 뚫고 중공소재로 만든 후 압연하거나 인발기 등로 인발해 소정이 외경과 두께로 만든 강관을 말하며소재에 구멍을 뚫은 다음 연신하는 경사압연방식과 이미 천공된 소재를 프레스로 압출하는 프레스방식등로 만듭니다. 예 심레스강관 접합 강관 - 단접강관용접강관용접법과 제조공정에 따라 세분화한다. 단접강관 소정의 폭으로 절단한 스켈프강관의 재로가 되는 반제품를 약 1400 ºC 정도로 가열한 후 연속식 단접기에서 관형태로 성형 동시에 양단을 롤로 압착시켜 제조합니다. 대부분이 소경관이며 대량 생산방식이므로 다품종 소량생산에는 어려움이 있습니다. 용접강관 대강 또는 스켈프를 상온에서 연속 롤 성형 통해 관형태로 성형 후 접합부를 용접해 제조하는 방법으로서 용접법에 따라 전기저항용접강관ERW강관가스용접관아크용접강관SAW강관으로 분류하며 SAW강관은 성형 따라 UOE 프레스강관스파이럴 강관롤벤딩 강관연속성형 구분합니다. UOE강관은 강력한 U형 프레스로 상온에서 전체길이를 U자형으로 구부리고O형 프레스로 성형 확관합니다. 장척 대경의 용접관 제조에 적합하고 성형 단순하며 용접길이가 짧아 능률이 높다는 특징이 있습니다. 스파이럴 조관법은 광폭 대강을 나선형으로 감아 원통형으로 성형 뒤 맞닿은 부분을 용접해 제조합니다. 동일폭의 강대를 사용해 나선 각도성형 각도를 변경함으로써 구경이 다른 강관을 제조할 수 있고 용접선이 나선형이기 때문에 응력이 비교적 낮습니다. 롤벤딩에 의한 조관법은 옛날부터 널리 사용된 방법이며 강관을 소경의 길이로 절단해 3~5개의 롤로 곡면 성형 원통형으로 만든뒤 맞닿은 양 edge를 용접하는 방법입니다.', 'Joining은 두 개 이상의 재료를 하나의 단위로 연결하는 과정입니다. 이 과정은 용접, 리벳, 브레이징, 접착 결합 등 다양한 기법을 통해 이루어지며, 사용되는 재료와 결합 부위의 강도에 따

[['강관을 제조방법별로 분류하면 접합부의 유무에 따라 무계목강관과 접합강관으로 크게 나누어집니다. 무계목 강관 빌릿 등의 소재를 가열해 천공기로 중심에 구멍을 뚫고 중공소재로 만든 후 압연하거나 인발기 등로 인발해 소정이 외경과 두께로 만든 강관을 말하며소재에 구멍을 뚫은 다음 연신하는 경사압연방식과 이미 천공된 소재를 프레스로 압출하는 프레스방식등로 만듭니다. 예 심레스강관 접합 강관 - 단접강관용접강관용접법과 제조공정에 따라 세분화한다. 단접강관 소정의 폭으로 절단한 스켈프강관의 재로가 되는 반제품를 약 1400 ºC 정도로 가열한 후 연속식 단접기에서 관형태로 성형 동시에 양단을 롤로 압착시켜 제조합니다. 대부분이 소경관이며 대량 생산방식이므로 다품종 소량생산에는 어려움이 있습니다. 용접강관 대강 또는 스켈프를 상온에서 연속 롤 성형 통해 관형태로 성형 후 접합부를 용접해 제조하는 방법으로서 용접법에 따라 전기저항용접강관ERW강관가스용접관아크용접강관SAW강관으로 분류하며 SAW강관은 성형 따라 UOE 프레스강관스파이럴 강관롤벤딩 강관연속성형 구분합니다. UOE강관은 강력한 U형 프레스로 상온에서 전체길이를 U자형으로 구부리고O형 프레스로 성형 확관합니다. 장척 대경의 용접관 제조에 적합하고 성형 단순하며 용접길이가 짧아 능률이 높다는 특징이 있습니다. 스파이럴 조관법은 광폭 대강을 나선형으로 감아 원통형으로 성형 뒤 맞닿은 부분을 용접해 제조합니다. 동일폭의 강대를 사용해 나선 각도성형 각도를 변경함으로써 구경이 다른 강관을 제조할 수 있고 용접선이 나선형이기 때문에 응력이 비교적 낮습니다. 롤벤딩에 의한 조관법은 옛날부터 널리 사용된 방법이며 강관을 소경의 길이로 절단해 3~5개의 롤로 곡면 성형 원통형으로 만든뒤 맞닿은 양 edge를 용접하는 방법입니다.',
  '무계목 강관과 접합 강관의 제조 방법을 비교할 때, 무계목 강관은 어떤 방식으로 소재에 구멍을 뚫고 연신하여 원하는 외경과 두께로 만드는지 설명하고, 접합 강관은 용접법에 따라 어떻게 분류